In [1]:
import sys, os
import signal, subprocess
import psutil

## Readme:

    https://www.linuxjournal.com/content/multiprocessing-python
    http://python-notes.curiousefficiency.org/en/latest/python3/multicore_python.html   
    https://opensource.com/article/17/4/grok-gil
    https://scipy-cookbook.readthedocs.io/items/ParallelProgramming.html

## psutil
https://psutil.readthedocs.io/en/latest

This python module is a cross-platform library for retrieving information on running processes and system utilization (CPU, memory, disks, network, sensors) in Python. It is useful mainly for system monitoring, profiling, limiting process resources and the management of running processes. It implements many functionalities offered by UNIX command line tools such as: ps, top, lsof, netstat, ifconfig, who, df, kill, free, nice, ionice, iostat, iotop, uptime, pidof, tty, taskset, pmap. psutil currently supports the following platforms:

 * Linux
 * Windows
 * macOS
 * FreeBSD, OpenBSD, NetBSD
 * Sun Solaris
 * AIX

In [8]:
print( psutil.cpu_stats(), '\n' )
print( psutil.cpu_times(), '\n' )
print( psutil.cpu_times_percent(interval=1, percpu=False), '\n' )
print( 'number of physical(?) CPUs: ', psutil.cpu_count(), '\n' )
print( psutil.cpu_times_percent(interval=1, percpu=True), '\n' )
print(sys.argv)

scpustats(ctx_switches=1810919006, interrupts=412877218, soft_interrupts=231547186, syscalls=0) 

scputimes(user=148641.94, nice=1127.24, system=42798.97, idle=1094404.32, iowait=533.35, irq=0.0, softirq=566.64, steal=0.0, guest=0.0, guest_nice=0.0) 

scputimes(user=2.5, nice=0.0, system=0.5, idle=97.0, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0) 

number of physical(?) CPUs:  8 

[scputimes(user=2.0, nice=0.0, system=0.0, idle=98.0, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0), scputimes(user=1.0, nice=0.0, system=0.0, idle=99.0, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0), scputimes(user=2.0, nice=0.0, system=0.0, idle=98.0, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0), scputimes(user=1.0, nice=0.0, system=0.0, idle=99.0, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0), scputimes(user=2.0, nice=0.0, system=1.0, idle=97.1, iowait=0.0, irq=0.0, softirq=0

In [50]:
def cpu_idle(nloop=999):
    """ simple example of signal handling
    """
    cpu_idle.flag = nloop
    
    def handler(signum, frame):
        print('Signal handler called with signal ',signum)
        #signal.alarm(0)
        cpu_idle.flag = 0
        
    signal.signal(signal.SIGINT, handler)

    while cpu_idle.flag > 0:
        cpu_idle.flag -= 1
        cpu = psutil.cpu_times_percent(interval=1, percpu=True)
        print('\t'.join([str(c.idle) for c in cpu]))

In [51]:
# make a simple CPU intensive task
#
cmnd = '[i**2 for i in range(9876543)]'
%timeit exec(cmnd)

3.45 s ± 32.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# spawn a subprocess to execute the task
#
status = subprocess.check_output('python -c "[i**2 for i in range(9876)]; print(True)"')
print('subprocess status: ', status)

In [11]:
cmnd = 'python -c "[i**2 for i in range(9876543)]; print(True)"'
cmnd = '/usr/bin/env python -c "[i**2 for i in range(9876543)]; print(True)"'


def paralize(cmdlist, nwait=5, nicer=False):
    
    plist = []
    for n, cmd in enumerate(cmdlist):
        plist.append( subprocess.Popen(cmnd, shell=True) ) 
#        print( plist[-1].pid )
#        if nicer and n >= psutil.cpu_count():
#            print('nice: ', psutil.Process( plist[-1].pid ).nice(-10) ) # set priority not supported under Windows

    for i in range(nwait):
        cpu = psutil.cpu_times_percent(interval=1, percpu=True)
        cpu = '\t'.join([str(c.idle) for c in cpu])
        active = ','.join( [str(n) for n,p in enumerate(plist) if p.poll() is None] )
        print(i, cpu, active)
        
    #for p in plist:
    #    print( psutil.Process(pid=p.pid) ).cpu_times()  # only works if process is alive...

# test for single subprocess
paralize([cmnd])

0 97.0	99.0	99.0	98.0	98.0	98.0	0.0	98.0 0
1 99.0	99.0	99.0	98.0	99.0	97.0	12.0	99.0 
2 99.0	99.0	99.0	98.0	97.0	98.0	97.0	98.0 
3 99.0	99.0	99.0	99.0	97.0	98.0	96.0	98.0 
4 99.0	100.0	99.0	99.0	98.0	98.0	96.0	99.0 


In [12]:
# run three subprocesses in parallel
#
paralize([cmnd for i in range(3)], nwait=5)

0 2.0	96.0	0.0	0.0	96.1	97.0	94.0	98.0 0,1,2
1 2.0	96.1	0.0	0.0	99.0	98.0	97.0	99.0 
2 99.0	96.0	97.0	99.0	96.0	97.0	95.0	97.0 
3 98.0	96.0	97.0	98.0	97.0	97.0	91.0	96.0 
4 98.0	99.0	100.0	98.0	98.0	97.0	96.0	96.0 
5 98.0	98.0	99.0	98.0	97.0	98.0	96.0	97.0 
6 97.0	98.0	96.0	98.0	96.0	97.0	96.0	97.0 


In [13]:
# run one process for each CPU
#
paralize([cmnd for i in range(8)], nwait=10)

0 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7
1 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7
2 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7
3 89.0	90.0	82.0	86.1	87.0	81.0	82.0	83.0 
4 97.0	98.0	98.0	99.0	98.0	98.0	97.0	99.0 
5 96.0	97.0	92.0	90.2	95.0	78.0	94.0	98.0 
6 96.0	97.0	99.0	96.0	97.0	89.0	95.0	94.0 
7 83.0	90.0	89.0	87.0	90.0	90.0	84.0	69.0 
8 75.0	99.0	97.0	96.1	96.0	96.0	97.0	98.0 
9 94.0	94.0	95.0	96.0	92.0	93.0	94.0	94.0 


In [14]:
# oversubscribing by 50% takes slightly longer
#
paralize([cmnd for i in range(12)], nwait=10)

0 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11
1 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11
2 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11
3 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11
4 24.0	15.8	29.0	23.0	13.9	27.0	29.0	27.0 
5 93.1	94.0	95.0	93.0	95.0	93.0	92.0	91.0 
6 88.0	93.0	95.0	92.1	93.0	95.0	91.0	89.0 
7 95.0	89.0	97.0	97.0	94.0	94.1	95.0	92.0 
8 97.0	98.0	99.0	99.0	97.0	98.0	97.0	97.0 
9 98.0	100.0	99.0	97.0	99.0	99.0	97.0	99.0 


In [15]:
paralize([cmnd for i in range(24)], nwait=15)

0 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
1 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
2 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
3 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
4 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
5 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
6 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
7 0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
8 0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0 7,12,15,17,20,21,22,23
9 82.0	84.0	89.0	88.0	92.0	84.0	86.1	93.0 
10 99.0	99.0	99.0	99.0	97.0	97.0	97.0	98.0 
11 99.0	100.0	99.0	98.0	98.0	97.0	95.0	98.0 
12 98.0	99.0	98.0	99.0	99.0	99.0	97.0	99.0 

#### Task: determine scaling efficiency
Gather estimates of computation as a function of number of processes, plot, and analyze.

Ideally we would have a perfectly linear relationship up to the number of CPUs and constant thereafter.  In practice the slope will be less than one and may saturate earlier.

## Multiprocessing
https://docs.python.org/3/library/multiprocessing.html?highlight=multiprocessing#module-multiprocessing

The multiprocessing package offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads. Due to this, the multiprocessing module allows the programmer to fully leverage multiple processors on a given machine. It runs on both Unix and Windows.

    from multiprocessing import Pool

    def f(x):
        return x*x

    if __name__ == '__main__':
        with Pool(5) as p:
            print(p.map(f, [1, 2, 3]))
    will print to standard output

    [1, 4, 9]
    
### Task: Use the multiprocessing package to speed up hyperparameter optimization.    

## Parallel python
https://www.parallelpython.com/       

### Figure out how to get this working on a single computer 

### Figure out how to get this working on all of the ST026 computers 